In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
sqlContext=SQLContext(sc)

##  La probabilidad de que la versión actual de un contenido fuera editada sin dejar comentario para usuarios que están logueados y que no están logueados

Cargo el parquet como DF y luego lo paso a RDD y verifico que se haya cargado bien

In [3]:
df_contents=sqlContext.read.parquet('contents.parquet')

In [4]:
rdd_contents=df_contents.rdd.cache()

In [5]:
rdd_contents.take(5)

[Row(title='Wikipedia:ArtÃ\xadculos solicitados', id=5, namespace=4, revision_id=132533307, parent_revision=132516631.0, revision_timestamp='2021-01-19T18:40:34Z', revisor_username='MetrÃ³nomo', revisor_id=1708233.0, revisor_ip=None, revisor_comment='AsÃ\xad esta mejor'),
 Row(title='Andorra', id=7, namespace=0, revision_id=138035057, parent_revision=138027475.0, revision_timestamp='2021-08-31T18:27:01Z', revisor_username=None, revisor_id=None, revisor_ip='2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2', revisor_comment='AÃ±adir contenido'),
 Row(title='Argentina', id=10, namespace=0, revision_id=137966826, parent_revision=137825165.0, revision_timestamp='2021-08-28T17:17:54Z', revisor_username='Traitrich', revisor_id=4642283.0, revisor_ip=None, revisor_comment='Reemplazos con [[Usuario:Benjavalero/Replacer|Replacer]]: Â«simbolosÂ»'),
 Row(title='GeografÃ\xada de Andorra', id=15, namespace=0, revision_id=137986788, parent_revision=133666383.0, revision_timestamp='2021-08-29T17:16:00Z', revisor

Como me pide la probabilidad de que la version actual  de un contenido fuera editada sin dejar comentarios para un usuario logueado y no logueado, calculo el total del RDD de contenido pues la probabilidad es igual a los casos favorables sobre los casos totales

In [6]:
total_contents = rdd_contents.count()

Hago un nuevo RDD con el map sobre el RDD de contenido dejando como clave el campo de usuario y como valor un identificador 'NC' si no hizo comentario o una 'C' si lo hizo

In [7]:
rdd_comments = rdd_contents.map(lambda x: (x[6], 'NC' if x[9]==None else 'C'))

Ahora hago dos RDD separados por usuarios y por no usuarios (no registrados). Hago dos filters, para los usuarios me fijo que la clave no sea None y que el valor sea igual a 'NC'. Para los no usuarios en el filter uso que la clave sea igual a None y el mismo filtro contra el valor.

In [8]:
rdd_comment_from_user = rdd_comments.filter(lambda x: x[0] != None and x[1]=='NC')
rdd_comment_from_NoUser = rdd_comments.filter(lambda x: x[0] == None and x[1]=='NC')

Hago el count de ambos RDD para poder calcular la probabilidad con el total que habia calculado antes

In [9]:
total_noComents_users = rdd_comment_from_user.count()

In [10]:
total_noComents_noUser = rdd_comment_from_NoUser.count()

Calculo las probabilidades. Prob= casosFavorables/casosTotales

In [11]:
prob_com_user = total_noComents_users/total_contents
prob_com_user

0.09490160603499764

In [12]:
prob_com_noUser = total_noComents_noUser/total_contents
prob_com_noUser

0.01865245425883387